# AfriSAR Search and Visualize

Authors: Nikita Susan (UAH), Aimee Barciauskas (DevSeed)

Date: January 17, 2023

Description:

## Run This Notebook
To access and run this tutorial within MAAP's Algorithm Development Environment (ADE), please refer to the ["Getting started with the MAAP"](https://docs.maap-project.org/en/latest/getting_started/getting_started.html) section of our documentation.

Disclaimer: it is highly recommended to run a tutorial within MAAP's ADE, which already includes packages specific to MAAP, such as maap-py. Running the tutorial outside of the MAAP ADE may lead to errors.

## About the Data

This dataset provides gridded estimates of aboveground biomass (AGB) for four sites in Gabon at 0.25 ha (50 m) resolution derived with field measurements and airborne LiDAR data collected from 2010 to 2016. The sites represent a mix of forested, savannah, and some agricultural and disturbed landcover types: Lope site, within Lope National Park; Mabounie, mostly forested site; Mondah Forest, protected area; and the Rabi forest site, part of the Smithsonian Institution of Global Earth Observatories world-wide network of forest plots. Plot-level biophysical measurements of tree diameter and tree height (or estimated by allometry) were performed at 1 ha and 0.25 ha scales on multiple plots at each site and used to derive AGB for each tree and then summed for each plot. Aerial LiDAR scans were used to construct digital elevation models (DEM) and digital surface models (DSM), and then the DEM and DSM were used to construct a canopy height model (CHM) at 1 m resolution. After checking site-plot locations against the CHM, mean canopy height (MCH) was computed over each 0.25 ha. A single regression model relating MCH and AGB estimates, incorporating local height based on the trunk DBH (HD) relationships, was produced for all sites and combined with the CHM layer to construct biomass maps at 0.25 ha resolution. (Source: [AfriSAR AGB User Guide](https://daac.ornl.gov/AFRISAR/guides/AfriSAR_AGB_Maps.html))

## Additional Resources
- [AfriSAR AGBD Dataset Landing Page](https://daac.ornl.gov/cgi-bin/dsviewer.pl?ds_id=1681)
- [Earthdata Search](https://search.earthdata.nasa.gov/search?q=AfriSAR_AGB_Maps_1681)

## Import and Install Packages

In [ ]:
# Uncomment and run the line below if missing packages
!pip install rioxarray hvplot

In [38]:
import rioxarray
import rasterio as rio
import hvplot.xarray
from maap.maap import MAAP
import boto3
from rasterio.session import AWSSession
import os

import warnings
warnings.filterwarnings("ignore")

## Search for AfriSAR AGB Data

Using MAAP's searchGranule function and the collection short name, we'll pull in the AfriSAR AGB collection.

In [8]:
maap = MAAP(maap_host='api.maap-project.org')

In [9]:
results = maap.searchGranule(cmr_host='cmr.earthdata.nasa.gov', short_name='AfriSAR_AGB_Maps_1681')
results

[{'concept-id': 'G2734344220-ORNL_CLOUD',
  'collection-concept-id': 'C2734261660-ORNL_CLOUD',
  'revision-id': '1',
  'format': 'application/echo10+xml',
  'Granule': {'GranuleUR': 'AfriSAR_AGB_Maps.Mondah_AGB_50m.tif',
   'InsertTime': '2022-11-28T00:00:00Z',
   'LastUpdate': '2023-07-17T18:24:45Z',
   'Collection': {'ShortName': 'AfriSAR_AGB_Maps_1681', 'VersionId': '1'},
   'DataGranule': {'DataGranuleSizeInBytes': '249846',
    'SizeMBDataGranule': '0.249846',
    'Checksum': {'Value': '2a7a7b99287f6a5aadd9792617763b5b9c2a140ca0bf116298421c06a7991ce3',
     'Algorithm': 'SHA-256'},
    'DayNightFlag': 'BOTH',
    'ProductionDateTime': '2022-11-28T00:00:00Z'},
   'Temporal': {'RangeDateTime': {'BeginningDateTime': '2016-02-01T00:00:00Z',
     'EndingDateTime': '2016-03-31T23:59:59Z'}},
   'Spatial': {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangle': {'WestBoundingCoordinate': '9.30308',
       'NorthBoundingCoordinate': '0.611119',
       'EastBoundingCoordinate': '9.46

Using the searchGranule function and the concept-id from our collection search, we can also discover granules within the collection.

In [47]:
granules = maap.searchGranule(
    cmr_host = 'cmr.earthdata.nasa.gov',
    concept_id = 'G2734344220-ORNL_CLOUD',
)
granules[0]

{'concept-id': 'G2734344220-ORNL_CLOUD',
 'collection-concept-id': 'C2734261660-ORNL_CLOUD',
 'revision-id': '1',
 'format': 'application/echo10+xml',
 'Granule': {'GranuleUR': 'AfriSAR_AGB_Maps.Mondah_AGB_50m.tif',
  'InsertTime': '2022-11-28T00:00:00Z',
  'LastUpdate': '2023-07-17T18:24:45Z',
  'Collection': {'ShortName': 'AfriSAR_AGB_Maps_1681', 'VersionId': '1'},
  'DataGranule': {'DataGranuleSizeInBytes': '249846',
   'SizeMBDataGranule': '0.249846',
   'Checksum': {'Value': '2a7a7b99287f6a5aadd9792617763b5b9c2a140ca0bf116298421c06a7991ce3',
    'Algorithm': 'SHA-256'},
   'DayNightFlag': 'BOTH',
   'ProductionDateTime': '2022-11-28T00:00:00Z'},
  'Temporal': {'RangeDateTime': {'BeginningDateTime': '2016-02-01T00:00:00Z',
    'EndingDateTime': '2016-03-31T23:59:59Z'}},
  'Spatial': {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangle': {'WestBoundingCoordinate': '9.30308',
      'NorthBoundingCoordinate': '0.611119',
      'EastBoundingCoordinate': '9.46755',
      'SouthB

## Adding items to an iterable

In [ ]:
items = [i.to_dict() for i in search.get_all_items()]
items[0]

## Extracting s3 link and reading it with rioxarray

In [48]:
granule_ur=granules[0]['Granule']['GranuleUR'].split(".")
collection_name=granule_ur[0]
file_name=granule_ur[1]

In [49]:
print(f"collection name: {collection_name} | file_name: {file_name}")

collection name: AfriSAR_AGB_Maps | file_name: Mondah_AGB_50m


In [50]:
def get_s3_creds(url):
    return maap.aws.earthdata_s3_credentials(url)

def get_s3_client(s3_cred_endpoint):
    creds=get_s3_creds(s3_cred_endpoint)
    boto3_session = boto3.Session(
            aws_access_key_id=creds['accessKeyId'],
            aws_secret_access_key=creds['secretAccessKey'],
            aws_session_token=creds['sessionToken']
    )
    return boto3_session.client("s3")

def download_s3_file(s3, bucket, collection_name, file_name):
    os.makedirs("/projects/afrisar", exist_ok=True) # create directories, as necessary
    download_path=f"/projects/afrisar/{file_name}.tif"
    s3.download_file(bucket, f"afrisar/{collection_name}/data/{file_name}.tif", download_path)
    return download_path

In [51]:
s3_cred_endpoint= 'https://data.ornldaac.earthdata.nasa.gov/s3credentials'
s3=get_s3_client(s3_cred_endpoint)

In [52]:
bucket="ornl-cumulus-prod-protected"
download_path=download_s3_file(s3, bucket, collection_name, file_name)
download_path

'/projects/afrisar/Mondah_AGB_50m.tif'

In [53]:
#item = items[0]
#s3_link = item['assets']['data']['href']
da = rioxarray.open_rasterio(download_path)
da = da.squeeze('band', drop=True)
da

<xarray.DataArray (y: 202, x: 366)>
[73932 values with dtype=float32]
Coordinates:
  * x            (x) float64 5.337e+05 5.338e+05 ... 5.519e+05 5.52e+05
  * y            (y) float64 6.752e+04 6.747e+04 ... 5.752e+04 5.747e+04
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     -9999.0
    scale_factor:   1.0
    add_offset:     0.0

## Visualizing the read data with hvplot

In [54]:
ds_masked = da.where(da != da._FillValue)

ds_masked.hvplot(
    'x', 'y', 
    cmap='viridis',
    frame_height=400,
    frame_width=400
).redim.range(value=(0,da.max().values))

:Image   [x,y]   (value)